# **Smiley prediction on Twitter data**

Learn more about the project [here](https://github.com/geofot96/MLProject2/)

### Before proceeding
Create a copy of this notebook by going to "File - Save a Copy in Drive"

### Setup environment
Make sure to change the runtime type to GPU under
Exécution > Modifier le type d'exécution 
To GPU and Mémoire RAM élevée

## 1.1 SET-UP: Set-up 1
Configuring Google Drive. 





In [1]:
# Mounting the Drive allows us to save and access our files
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
import os
os.chdir("/content/drive/My Drive")

In [3]:
!git clone https://paola-md:<password>@github.com/geofot96/MLProject2

Cloning into 'MLProject2'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 311 (delta 205), reused 254 (delta 159), pack-reused 0
Receiving objects: 100% (311/311), 167.56 KiB | 5.24 MiB/s, done.
Resolving deltas: 100% (205/205), done.


In [17]:
# Make sure we have the files
import os
os.chdir("/content/drive/My Drive/MLProject2")
os.listdir()

['.git',
 '.gitignore',
 'Dockerfile-notebook',
 'README.md',
 'build_vocab.sh',
 'cooc.py',
 'cut_vocab.sh',
 'glove_template.py',
 'pickle_vocab.py',
 'project2_description.pdf',
 'requirements.txt',
 'src',
 'test',
 'data',
 'models']

In [5]:
!mkdir data
!mkdir models
!mkdir predictions

## 1.2 SET-UP STEP 2:
Download the data from [here](https://www.aicrowd.com/challenges/epfl-ml-text-classification/dataset_files) and place it in the newly created folder data

In [10]:
!unzip data/twitter-datasets.zip -d data/

Archive:  CS433-ML-project2/bert/data/twitter-datasets.zip
  inflating: MLProject2/data/twitter-datasets/sample_submission.csv  
  inflating: MLProject2/data/twitter-datasets/test_data.txt  
  inflating: MLProject2/data/twitter-datasets/train_neg_full.txt  
  inflating: MLProject2/data/twitter-datasets/train_neg.txt  
  inflating: MLProject2/data/twitter-datasets/train_pos_full.txt  
  inflating: MLProject2/data/twitter-datasets/train_pos.txt  


In [19]:
!mv data/twitter-datasets/train_neg.txt data/train_neg.txt 
!mv data/twitter-datasets/train_pos.txt data/train_pos.txt 
!mv data/twitter-datasets/train_neg_full.txt data/train_neg_full.txt 
!mv data/twitter-datasets/train_pos_full.txt data/train_pos_full.txt 
!mv data/twitter-datasets/test_data.txt data/test_data.txt

## 1.3 SET-UP STEP 3
Install the required dependencies

In [4]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install emoji
!pip install unidecode

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-4d7zt7np
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-4d7zt7np
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 13.4MB/s 
     |████████████████████████████████| 2.9MB 34.4MB/s 
  Created wheel for transformers: filename=transformers-4.1.0.dev0-cp36-none-any.whl size=1450263 sha256=ace9e0a2b71910d88b866cab4a78ccb1e7a3a66b959adafa407aa099c687e5f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-g6vxhzcb/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=adef8d222f224843fe9662d22a96e6bf03e63e7427a5d3630942aa24d366aa87
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a5012

## 1.4 SET-UP STEP 4

Import the necessary files

In [19]:
import sys
sys.path.append("/content/drive/My Drive/MLProject2")

In [20]:
import pandas as pd
%reload_ext autoreload
%autoreload 2
from src.preprocessing import apply_preprocessing, apply_preprocessing_bert
from src.data_loading import load_tweets, load_test_tweets, split_data, seed_everything, split_data_bert
from src.models.bert import run_bert, predict_bert

In [21]:
os.chdir("/content/drive/My Drive/MLProject2/src")

## 2.1 MODELING STEP 1
Loading tweets


In [8]:
# Global variables
model_name = 'digitalepidemiologylab/covid-twitter-bert'
seed_everything()

In [9]:
#tweets = load_tweets(sample=False, frac=1)
tweets = load_tweets(sample=True, frac=0.0001)

Positive tweets: 10
Negative tweets: 10
Most frequent label model: 0.503


In [10]:
# Explore loaded tweets
tweets.head()

,tweet,polarity
0,beyonce 14x26 custom picture frame / poster fr...,0
1,<user> never dodgers and angels,1
2,<user> cmon on then were are ya ? utt lol,1
3,"classic chocolate covered matzos , 7 - ounce b...",0
4,i want an ipod with a camera,0


## 2.2 MODELING STEP 2
Preprocess tweets
(The tweets are tokenized)


In [11]:
tweets = apply_preprocessing_bert(tweets)

## 2.3 MODELING STEP 3
Train the model



In [12]:
train_tweets, val_tweets = split_data_bert(tweets)


In [13]:
run_bert(train_tweets=train_tweets,
          val_tweets=val_tweets,
          learning_rate = 5e-6,
          model_name = 'bert',
          epochs = 2)


----------------------------------------------------------------------
MODEL TO RUN: Neural network with bert tokens
----------------------------------------------------------------------

Creating batches...


Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifi

Saving model...


Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

../../models/bert_0
Done!


...

EPOCH: 0. Losses: train = 0.6863088011741638, val = 0.4405183792114258.             Accuracy: 1.0


Iteration: 100%|██████████| 1/1 [00:00<00:00,  2.45it/s]


Saving model...


Iteration: 100%|██████████| 1/1 [00:00<00:00, 36.01it/s]

../../models/bert_1
Done!
EPOCH: 1. Losses: train = 0.6271177530288696, val = 0.4793529808521271.             Accuracy: 1.0


## 2.4 MODELING STEP 4
Use the trained model to predict

In [15]:
test_tweets = load_test_tweets()
test_tweets['polarity'] =  pd.to_numeric(test_tweets.id)  # to use same preprocessing function
dataset = apply_preprocessing_bert(test_tweets)

In [16]:

test_ids_list, binary_preds_list = predict_bert(dataset, 'bert_0')

binary_preds_list[binary_preds_list == 0] = -1

results = pd.DataFrame({'Id': test_ids_list, 'Prediction': binary_preds_list})
results.to_csv("./../predictions/predictions.csv", index=False)



Iteration: 100%|██████████| 312/312 [00:54<00:00,  5.69it/s]


FileNotFoundError: ignored